In [1]:
import os

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

gpu_on = True

if gpu_on :
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from gan_module.data_loader.classification import ClassifyDataloader
from glob import glob

data_set_name = "concreate_crack"
batch_size = 32

train_image_path = glob(f"./datasets/{data_set_name}/*/*.jpg")
label_list = os.listdir(f"./datasets/{data_set_name}/")
label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}
train_data_loader = ClassifyDataloader(image_path_list=train_image_path,
                                       label_to_index_dict = label_to_index_dict,
                                       batch_size = 32,
                                       shuffle=True
)

In [3]:
from tensorflow.keras.applications import InceptionV3, ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D 
from tensorflow.keras.activations import sigmoid

loss_function = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
base_model = ResNet50V2(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=train_data_loader.source_data_shape,
    classes=None,
    pooling=None,
    classifier_activation=None
)

gap_passed = GlobalAveragePooling2D()(base_model.output)
dense_passed = Dense(units=train_data_loader.num_classes, activation="sigmoid")(gap_passed)

model = Model(base_model.inputs, dense_passed)

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import Nadam

optimizer = Nadam(1e-3, clipnorm=1)

# save_c = ModelCheckpoint(
#     "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
#     monitor='val_loss',
#     verbose=0,
#     save_best_only=False,
#     save_weights_only=True,
#     mode='min')

save_c = ModelCheckpoint(
    "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-9)
csv_logger = CSVLogger('./U_net/log.csv', append=False, separator=',')

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
start_epoch = 0
epochs = 200

model.fit(
    train_data_loader,
#     validation_data=self.valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger],
    initial_epoch=start_epoch
)

Epoch 1/200
1250/1250 [==============================] - 338s 258ms/step - loss: 0.1826 - accuracy: 0.9656
Epoch 2/200
1250/1250 [==============================] - 323s 259ms/step - loss: 0.1316 - accuracy: 0.9983
Epoch 3/200
1250/1250 [==============================] - 322s 257ms/step - loss: 0.1274 - accuracy: 0.9981
Epoch 4/200
1250/1250 [==============================] - 318s 254ms/step - loss: 0.1216 - accuracy: 0.9998
Epoch 5/200
1250/1250 [==============================] - 319s 255ms/step - loss: 0.1214 - accuracy: 0.9998
Epoch 6/200
1250/1250 [==============================] - 319s 255ms/step - loss: 0.1201 - accuracy: 0.9998
Epoch 7/200
1250/1250 [==============================] - 318s 254ms/step - loss: 0.1224 - accuracy: 0.9985
Epoch 8/200
1250/1250 [==============================] - 318s 254ms/step - loss: 0.1194 - accuracy: 1.0000
Epoch 9/200
1250/1250 [==============================] - 318s 254ms/step - loss: 0.1198 - accuracy: 0.9998
Epoch 10/200
1250/1250 [=============